In [11]:
from IsingLib import Model, Hamil, DaSampler
import math
import itertools

import numpy as np
from pyqubo import Array
import random
import itertools

In [12]:
def k3_cal(city, lis):
    dis_lis = []
    for s, coord_s in lis.items():
        for t, coord_t in lis.items():
            if s < t:
                x_s, y_s = coord_s
                x_t, y_t = coord_t
                dist = math.sqrt((x_s - x_t) ** 2 + (y_s - y_t) ** 2)
                dis_lis.append(int(dist))
            else:
                pass
    return makeDistanceMatrix(city, dis_lis)


# cityNum:都市数 int, ls:EDGE_WEIGHT_SECTIONのリスト
def makeDistanceMatrix(city, ls):
    # cityNum*cityNumの配列の作成
    mat = np.zeros((city, city))
    # 代入するx,yの組み合わせの作成
    target = list(itertools.combinations(range(city), 2))
    for ((x, y), dist) in zip(target, ls):
        mat[x][y] = dist
    return mat


def make_mat(fname):
    with open(fname, 'r') as f:
        line = f.read()
    line_li = line.split('\n')
    i = 0
    params = {}
    edges = []
    coord = {}
    demand = {}
    key = ''
    while len(line_li) > i:
        if key == 'EDGE_WEIGHT_SECTION':
            e = line_li[i].split()
            if e[0] == 'DEMAND_SECTION':
                params[key] = edges
                key = e[0]
            else:
                edges += map(int, e)
        elif key == 'NODE_COORD_SECTION':
            c = line_li[i].split()
            if c[0] == 'DEMAND_SECTION':
                params[key] = coord
                key = c[0]
            else:
                coord[int(c[0])] = (int(c[1]), int(c[2]))
        elif key == 'DEMAND_SECTION':
            d = line_li[i].split()
            if d[0] == 'DEPOT_SECTION':
                params[key] = demand
                key = d[0]
            else:
                demand[int(d[0])] = int(d[1])
        elif key == 'DEPOT_SECTION':
            params[key] = int(line_li[i])
            key = ''
        else:
            line_tmp = line_li[i].split(':')
            key = line_tmp[0].strip()
            if len(line_tmp) > 1:
                params[key] = line_tmp[1].strip()
        i += 1

    city = int(params['DIMENSION'])
    if 'EDGE_WEIGHT_SECTION' in params:
        ls = params['EDGE_WEIGHT_SECTION']
        mat = makeDistanceMatrix(city, ls)
    elif 'NODE_COORD_SECTION' in params:
        ls = params['NODE_COORD_SECTION']
        mat = k3_cal(city, ls)
    return mat, city, int(params['CAPACITY']), list(params['DEMAND_SECTION'].values())


In [13]:
#mat, city = make_mat("32-5.vrp")

In [14]:
mat, city, capacity, weights = make_mat("test.vrp")
print(mat, city, capacity, weights)

[[ 0.  9. 14. 21. 23. 22. 25. 32. 36. 38. 42. 50. 52.]
 [ 0.  0.  5. 12. 22. 21. 24. 31. 35. 37. 41. 49. 51.]
 [ 0.  0.  0.  7. 17. 16. 23. 26. 30. 36. 36. 44. 46.]
 [ 0.  0.  0.  0. 10. 21. 30. 27. 37. 43. 31. 37. 39.]
 [ 0.  0.  0.  0.  0. 19. 28. 25. 35. 41. 29. 31. 29.]
 [ 0.  0.  0.  0.  0.  0.  9. 10. 16. 22. 20. 28. 30.]
 [ 0.  0.  0.  0.  0.  0.  0.  7. 11. 13. 17. 25. 27.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 10. 16. 10. 18. 20.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  6.  6. 14. 16.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 12. 12. 20.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  8. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]] 13 60 [0, 12, 17, 15, 14, 17, 14, 12, 19, 18, 16, 17, 11]


In [ ]:
M = city #city # 都市数 (実際はファイルから読み取る)
N = 4 # トラック数 (実際はファイルから読み取る)
Q = capacity # トラックの容量制限 (実際はファイルから読み取る)

#max_weight = 5 # 重さの最大値 (実際はファイルから読み取る)
#max_distance = 10 # 距離の最大値 (実際はファイルから読み取る)
vartype='BINARY'
x = Array.create('x', shape=(M, M, N), vartype=vartype)
y = Array.create('y', shape=(M, N), vartype=vartype)
s = Array.create('s', shape=(M, Q), vartype=vartype)
#h = random.sample(range(max_weight), M) # 都市数分の荷物の重さを生成 (実際はファイルから読み取る)
h = weights

#都市間の距離を生成 (実際はファイルから読み取る)
#d = [[0] * M for _ in range(M)]
#target = list(itertools.combinations(range(M), 2))
#for a, b in target:
#    d[a][b] = random.randint(0, max_distance)
d = mat


# In[118]:


# 制約1
def H_11():
    def H_T(p):
        return sum((( sum(x[c][t][p] for c in range(M)) - 1/2)**2) for t in range(M))
    
    def H_C(p):
        return sum((( sum(x[c][t][p] for t in range(M)) - 1/2)**2) for c in range(M))
    
    def h_v(p):
        ht = H_T(p)
        hc = H_C(p)
        print(ht)
        print(hc)
        return  ht + hc
    
    return sum(h_v(v) for v in range(N))


# In[119]:


# 制約2
def H_12():
    def H_V(k):
        return sum((sum(x[c][k][v] for c in range(M))-0.5)**2 for v in range(N))
    
    def H_C(k):
        return sum((sum(x[c][k][v] for v in range(N))-0.5)**2 for c in range(M))
    
    def h_T(k):
        return H_V(k)+H_C(k)
    
    return sum(h_T(t) for t in range(M))


# In[120]:


# 制約3
def H_13():
    def H_v(i):
        return sum((( sum(x[i][t][v] for t in range(M)) - 1/2)**2) for v in range(N))

    def H_T(i):
        return sum((( sum(x[i][t][v] for v in range(N)) - 1/2)**2) for t in range(M))

    def h_c(i):
        return H_v(i) + H_T(i)
    
    return sum(h_c(c) for c in range(M))


# In[121]:


# 制約4
def H_14():
    def h_y(p):
        return sum(( (sum(x[c][t][p] - y[c][p] for t in range(M)))**2) for c in range(M))
    
    return sum(h_y(v) for v in range(N))


# In[122]:


# 制約5 は制約4と同じ


# In[123]:


# 制約6
def H_16():
    def h_Q(p):
        h_Q1 = (1-sum(s[p][n] for n in range(Q)))**2
        h_Q2 = sum(N*s[p][n] for n in range(Q))
        h_Q3 = sum(sum(h[c]*y[c][p] for t in range(M)) for c in range(M))
        return h_Q1 + (h_Q2-h_Q3)**2
    
    return sum(h_Q(v) for v in range(N))


# In[124]:


# 制約7
def H_17():
    def h_Q(p):
        return (1 - sum(s[p][n] for n in range(Q)))**2 + (sum(n * s[p][n] for n in range(Q)) - sum( (sum(h[c] * x[c][t][p] for t in range(M))) for c in range(M)))**2

    return sum(h_Q(v) for v in range(N))


# In[125]:


# 目的式
def H_21():
    def h_d(p):
        return sum( (sum( (sum( d[i][j] * x[i][t][p] * x[j][(t+1) % M][p] for j in range(M) )) for i in range(M)) ) for t in range(M) )
    
    return sum(h_d(v) for v in range(N))

In [5]:
h11 = Model.create('q', H_11())
# + H_12() + H_13()  + H_17() + H_21()

NameError: name 'H_11' is not defined

In [65]:
h_c1 = Model.create(vartype, H_11())
print(h_c1)

(((Binary(x[0][0][0])+Binary(x[1][0][0])+Binary(x[2][0][0])+Num(-0.5))*(Binary(x[0][0][0])+Binary(x[1][0][0])+Binary(x[2][0][0])+Num(-0.5)))+((Binary(x[0][1][0])+Binary(x[1][1][0])+Binary(x[2][1][0])+Num(-0.5))*(Binary(x[0][1][0])+Binary(x[1][1][0])+Binary(x[2][1][0])+Num(-0.5)))+((Binary(x[0][2][0])+Binary(x[1][2][0])+Binary(x[2][2][0])+Num(-0.5))*(Binary(x[0][2][0])+Binary(x[1][2][0])+Binary(x[2][2][0])+Num(-0.5))))
(((Binary(x[0][0][0])+Binary(x[0][1][0])+Binary(x[0][2][0])+Num(-0.5))*(Binary(x[0][0][0])+Binary(x[0][1][0])+Binary(x[0][2][0])+Num(-0.5)))+((Binary(x[1][0][0])+Binary(x[1][1][0])+Binary(x[1][2][0])+Num(-0.5))*(Binary(x[1][0][0])+Binary(x[1][1][0])+Binary(x[1][2][0])+Num(-0.5)))+((Binary(x[2][0][0])+Binary(x[2][1][0])+Binary(x[2][2][0])+Num(-0.5))*(Binary(x[2][0][0])+Binary(x[2][1][0])+Binary(x[2][2][0])+Num(-0.5))))
(((Binary(x[0][0][1])+Binary(x[1][0][1])+Binary(x[2][0][1])+Num(-0.5))*(Binary(x[0][0][1])+Binary(x[1][0][1])+Binary(x[2][0][1])+Num(-0.5)))+((Binary(x[0][1

In [61]:
import numpy
numpy.any(h_c1.matrix == 1)

False

In [66]:
H_11().compile()

(((Binary(x[0][0][0])+Binary(x[1][0][0])+Binary(x[2][0][0])+Num(-0.5))*(Binary(x[0][0][0])+Binary(x[1][0][0])+Binary(x[2][0][0])+Num(-0.5)))+((Binary(x[0][1][0])+Binary(x[1][1][0])+Binary(x[2][1][0])+Num(-0.5))*(Binary(x[0][1][0])+Binary(x[1][1][0])+Binary(x[2][1][0])+Num(-0.5)))+((Binary(x[0][2][0])+Binary(x[1][2][0])+Binary(x[2][2][0])+Num(-0.5))*(Binary(x[0][2][0])+Binary(x[1][2][0])+Binary(x[2][2][0])+Num(-0.5))))
(((Binary(x[0][0][0])+Binary(x[0][1][0])+Binary(x[0][2][0])+Num(-0.5))*(Binary(x[0][0][0])+Binary(x[0][1][0])+Binary(x[0][2][0])+Num(-0.5)))+((Binary(x[1][0][0])+Binary(x[1][1][0])+Binary(x[1][2][0])+Num(-0.5))*(Binary(x[1][0][0])+Binary(x[1][1][0])+Binary(x[1][2][0])+Num(-0.5)))+((Binary(x[2][0][0])+Binary(x[2][1][0])+Binary(x[2][2][0])+Num(-0.5))*(Binary(x[2][0][0])+Binary(x[2][1][0])+Binary(x[2][2][0])+Num(-0.5))))
(((Binary(x[0][0][1])+Binary(x[1][0][1])+Binary(x[2][0][1])+Num(-0.5))*(Binary(x[0][0][1])+Binary(x[1][0][1])+Binary(x[2][0][1])+Num(-0.5)))+((Binary(x[0][1

Model(CompiledQubo({('x[0][0][0]', 'x[0][0][0]'): 0.0,
 ('x[0][0][0]', 'x[0][1][0]'): 2.0,
 ('x[0][0][0]', 'x[0][2][0]'): 2.0,
 ('x[0][0][0]', 'x[1][0][0]'): 2.0,
 ('x[0][0][0]', 'x[2][0][0]'): 2.0,
 ('x[0][0][1]', 'x[0][0][1]'): 0.0,
 ('x[0][0][1]', 'x[0][1][1]'): 2.0,
 ('x[0][0][1]', 'x[0][2][1]'): 2.0,
 ('x[0][0][1]', 'x[1][0][1]'): 2.0,
 ('x[0][0][1]', 'x[2][0][1]'): 2.0,
 ('x[0][1][0]', 'x[0][1][0]'): 0.0,
 ('x[0][1][0]', 'x[0][2][0]'): 2.0,
 ('x[0][1][0]', 'x[1][1][0]'): 2.0,
 ('x[0][1][0]', 'x[2][1][0]'): 2.0,
 ('x[0][1][1]', 'x[0][1][1]'): 0.0,
 ('x[0][1][1]', 'x[0][2][1]'): 2.0,
 ('x[0][1][1]', 'x[1][1][1]'): 2.0,
 ('x[0][1][1]', 'x[2][1][1]'): 2.0,
 ('x[0][2][0]', 'x[0][2][0]'): 0.0,
 ('x[0][2][0]', 'x[1][2][0]'): 2.0,
 ('x[0][2][0]', 'x[2][2][0]'): 2.0,
 ('x[0][2][1]', 'x[0][2][1]'): 0.0,
 ('x[0][2][1]', 'x[1][2][1]'): 2.0,
 ('x[0][2][1]', 'x[2][2][1]'): 2.0,
 ('x[1][0][0]', 'x[1][0][0]'): 0.0,
 ('x[1][0][0]', 'x[1][1][0]'): 2.0,
 ('x[1][0][0]', 'x[1][2][0]'): 2.0,
 ('x[1][0